In [15]:
%load_ext autoreload
%autoreload 2

from os import makedirs
from os.path import exists
from shutil import copy
from sqlite3 import connect
from baboon_tracking import BaboonTracker
from library.dataset import get_dataset_path, get_dataset_list
from library.region import bb_intersection_over_union

import numpy as np
import pandas as pd

runtime_config = {"display": False, "save": True, "timings": False}

video_files = [
    "VISO/car/003", # Video 1
    "VISO/car/009", # Video 7
]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
# video_files = get_dataset_list("VISO")

for video_file in video_files:
    results_path = f"./output/results/{video_file}/results.db"
    if exists(results_path):
        continue

    path = get_dataset_path(video_file)

    baboon_tracker = BaboonTracker(path, runtime_config=runtime_config)
    baboon_tracker.run()

    makedirs(f"./output/results/{video_file}", exist_ok=True)
    copy("./output/results.db", results_path)

https://e4e-nas.ucsd.edu:5001/webapi/auth.cgi?api=SYNO.API.Auth
{'data': {'sid': 'HzbXdiP118pcwkYH4AachMmjteQKYetxnwllR-1ZVQsDNJ1kMfTVSXICQBoFpQqycfz-kChKrT9f0alW7Huoig'}, 'success': True}


 31%|███       | 92/300 [00:23<00:55,  3.74it/s]

In [ ]:
for video_file in video_files:
    file_name = f"./output/results/{video_file}/results.db"

    with connect(file_name) as connection:
        cursor = connection.cursor()

        ground_truth = pd.read_csv(f"./data/Datasets/{video_file}/gt/gt.txt").to_numpy()
        found_regions = cursor.execute(
            "SELECT x1, y1, x2, y2, frame FROM motion_regions"
        )

        curr_frame = -1
        true_positive = 0
        false_negative = 0
        false_positive = 0
        truth = None
        for x1, y1, x2, y2, frame in found_regions:
            if curr_frame != frame:
                if truth is not None:
                    false_negative += truth.shape[0]

                truth = np.array(
                    [
                        (x1, y1, x1 + width, y1 + height)
                        for x1, y1, width, height in ground_truth[
                            ground_truth[:, 0] == frame, 2:6
                        ]
                    ]
                )
                curr_frame = frame

            current = (x1, y1, x2, y2)
            matches = np.array([bb_intersection_over_union(current, t) for t in truth])

            if np.sum(matches) > 0:
                true_positive += 1
                truth = truth[np.logical_not(matches == np.max(matches))]
            else:
                false_positive += 1

        recall = true_positive / (false_negative + true_positive)
        precision = true_positive / (true_positive + false_positive)
        f1 = (2 * recall * precision) / (recall + precision)

        print(f"Recall: {recall}, Precision: {precision}, F1: {f1}")

Recall: 0.6029016321680946, Precision: 0.19132764437388372, F1: 0.2904746840210301
